In [ ]:
# Info: Quite often, we need to know the freqiencies of mutations rising to high frequencies
# in an experiment. Given that we have a fasta file that is built from all variants, this code
# calculates the freqiencies of single point mutations and outputs them as R objects 

In [1]:
# Load the relevant libraries
library("Biostrings")
library("seqinr")
library("stringr")
library("ggplot2")
library("reshape")

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attaching package: ‘Biostrings’


The following object is masked from ‘package:base’:



In [2]:
## All necessary and imporant functions to load
### This function calculated the frequency of mutants in a population

read_freq <- function (mut_data, all_mutants){

mutants_freq=matrix(0, nrow=length(mut_data), ncol=length(all_mutants))

for (i in 1:length(mut_data)){    
    index_row=match(mut_data[[i]], all_mutants)        
    mutants_freq[i,index_row]=1
}
return(colSums(mutants_freq)/length(mut_data))
}


## Serial version of the code

In [ ]:
start_time <- Sys.time()

# This function checkes a sequence of interest against a WT sequenec and 
# and prints all the single point mutations that are different between the two


diff_seq <- function(num){
    
    ## Output_list
    mutant_list=list()
    
    ## Load libraries
    library("Biostrings")
    library("seqinr")
    
    # Load data
    mutants_seq=readAAStringSet("All_fasta_files/INC_NAR_1_1_AA.fasta")
    WT_seq=readAAStringSet("All_fasta_files/TetR_wt_AA.fasta")

   
    pair=pairwiseAlignment(WT_seq, mutants_seq[num], type="local-global" )
    pair_sum=summary(pair)
    sum_table <- pair_sum@mismatchSummary$subject
    
    mutant_list=paste(sum_table$Pattern,"_",sum_table$SubjectPosition, "_", sum_table$Subject,sep="")

    
    return(mutant_list)
       
    }

## Parallel version of the code

In [ ]:
library("parallel")

In [ ]:
limit=length(mutants_seq)

start_time <- Sys.time()

# create cluster object
clust <- makeCluster(24)


# test each number in sample_numbers for primality
result_pl <- parSapply(cl = clust, 1:limit, FUN = diff_seq)

# close cluster object
stopCluster(clust)

end_time <- Sys.time()

end_time - start_time

In [ ]:
unique(unlist(result_pl))

## Counting mutations in all populations (parallel processing)

In [ ]:
All_fasta_files <- list.files("./All_fasta_files/", pattern = "_AA.fasta")

In [ ]:
WT_seq=readAAStringSet("All_fasta_files/TetR_wt_AA.fasta")


## Define the population level function
for (i in 1:length(All_fasta_files)){
    
 pop_file=paste("All_fasta_files/", All_fasta_files[i], sep="")  
 mutants_seq= readAAStringSet(filepath = pop_file)
 
 ## Write a space variable to read from
 
 saveRDS(object = mutants_seq, file = "mutants_seq.RData")

 
 diff_seq <- function(num){
    
    
    ## Load libraries
    library("Biostrings")
    library("seqinr")
    
    WT_seq=readAAStringSet("All_fasta_files/TetR_wt_AA.fasta")
    mutants_seq=readRDS("mutants_seq.RData")
   
    pair=pairwiseAlignment(WT_seq, mutants_seq[num], type="local-global" )
    pair_sum=summary(pair)
    sum_table <- pair_sum@mismatchSummary$subject
    
    mutant_list=paste(sum_table$Pattern,"_",sum_table$SubjectPosition, "_", sum_table$Subject,sep="")

    
    return(mutant_list)
       
    }

## Run the loop
limit=length(mutants_seq)


# create cluster object
clust <- makeCluster(24)

    
# test each number in sample_numbers for primality
result_pl <- parSapply(cl = clust, 1:limit, FUN = diff_seq)

# close cluster object
stopCluster(clust)

saveRDS(result_pl, file=paste("./Mutational_freqs_lists/", All_fasta_files[i]))
    
}

### Getting the frequency data

In [ ]:
files=list.files("./Mutational_freqs_lists/")
mutant_list=vector()

for (i in 1:length(files)){
    
    
    mut_data=readRDS(paste("./Mutational_freqs_lists/", files[i], sep=""))
    
    mutant_list=c(mutant_list, unique(unlist(mut_data)))
    
    
    }

all_mutants=unique(mutant_list)

In [ ]:
mutants_frequencies=matrix(NA, nrow =length(files) , ncol =length(all_mutants) )
colnames(mutants_frequencies)=all_mutants

for (i in 1:length(files)){

    ### All_mutants in the population
    mut_data=readRDS(paste("./Mutational_freqs_lists/", files[i], sep=""))
    
    ### Frequency
    mutant_table=table(unlist(mut_data))    
    mutant_table_sorted=sort(mutant_table, decreasing = TRUE)
    
    ## Invoke the following command if you'd like to exclude WT from the analysis
    #mutant_table_sorted=mutant_table_sorted[-which(names(mutant_table_sorted)=="__")]
    
    mutant_freq=mutant_table_sorted/sum(mutant_table_sorted)

      
    index_row=match(names(mutant_freq), all_mutants)
    mutants_frequencies[i,index_row]=mutant_freq
    
    message(paste("protein=", as.character(i), "|from", as.character(length(files))))
    
    }